In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parents[0]
sys.path.append(str(PROJECT_ROOT))

In [2]:
import pandas as pd
from src.db import get_engine

In [3]:
from src.transform import (
    add_moving_averages,
    add_daily_returns,
    add_rsi
)

In [4]:
engine = get_engine(
    user="postgres",
    password="admin",
    host="localhost",
    port=5432,
    db="stock_data"
)

In [5]:
df = pd.read_sql("""
    SELECT *
    FROM daily_prices
    WHERE ticker = 'TLKM.JK'
    ORDER BY date
""", engine)

In [6]:
df = df.sort_values("date")

In [7]:
display(df.head(), df.tail())

,id,ticker,date,open,high,low,close,volume,created_at
0,1,TLKM.JK,2004-09-28,345.4697,345.4697,341.3324,341.3324,90095000,2026-01-01 15:07:38.819122
1,2,TLKM.JK,2004-09-29,341.3324,341.3324,335.1263,341.3324,29370000,2026-01-01 15:07:38.819122
2,3,TLKM.JK,2004-09-30,341.3325,343.4012,341.3325,343.4012,42927500,2026-01-01 15:07:38.819122
3,4,TLKM.JK,2004-10-01,343.4010,349.6070,341.3323,347.5384,108387500,2026-01-01 15:07:38.819122
4,5,TLKM.JK,2004-10-04,349.6071,364.0879,349.6071,359.9505,424210000,2026-01-01 15:07:38.819122


,id,ticker,date,open,high,low,close,volume,created_at
5236,5237,TLKM.JK,2025-12-22,3440.0,3490.0,3420.0,3470.0,113720300,2026-01-01 15:07:38.819122
5237,5238,TLKM.JK,2025-12-23,3470.0,3470.0,3420.0,3470.0,39083700,2026-01-01 15:07:38.819122
5238,5239,TLKM.JK,2025-12-24,3480.0,3500.0,3440.0,3460.0,30919800,2026-01-01 15:07:38.819122
5239,5240,TLKM.JK,2025-12-29,3480.0,3530.0,3460.0,3490.0,53079900,2026-01-01 15:07:38.819122
5240,5241,TLKM.JK,2025-12-30,3490.0,3510.0,3480.0,3480.0,48019100,2026-01-01 15:07:38.819122


In [8]:
df = add_moving_averages(df, price_col="close")
df = add_daily_returns(df, price_col="close")
df = add_rsi(df, price_col="close")

In [9]:
display(df.head(), df.tail())

,id,ticker,date,open,high,low,close,volume,created_at,ma_5,ma_20,ma_50,daily_return,rsi
0,1,TLKM.JK,2004-09-28,345.4697,345.4697,341.3324,341.3324,90095000,2026-01-01 15:07:38.819122,NaN,NaN,NaN,NaN,NaN
1,2,TLKM.JK,2004-09-29,341.3324,341.3324,335.1263,341.3324,29370000,2026-01-01 15:07:38.819122,NaN,NaN,NaN,0.000000,NaN
2,3,TLKM.JK,2004-09-30,341.3325,343.4012,341.3325,343.4012,42927500,2026-01-01 15:07:38.819122,NaN,NaN,NaN,0.006061,NaN
3,4,TLKM.JK,2004-10-01,343.4010,349.6070,341.3323,347.5384,108387500,2026-01-01 15:07:38.819122,NaN,NaN,NaN,0.012048,NaN
4,5,TLKM.JK,2004-10-04,349.6071,364.0879,349.6071,359.9505,424210000,2026-01-01 15:07:38.819122,346.71098,NaN,NaN,0.035714,NaN


,id,ticker,date,open,high,low,close,volume,created_at,ma_5,ma_20,ma_50,daily_return,rsi
5236,5237,TLKM.JK,2025-12-22,3440.0,3490.0,3420.0,3470.0,113720300,2026-01-01 15:07:38.819122,3472.0,3568.0,3447.2,0.017595,43.835616
5237,5238,TLKM.JK,2025-12-23,3470.0,3470.0,3420.0,3470.0,39083700,2026-01-01 15:07:38.819122,3462.0,3559.0,3457.4,0.000000,42.253521
5238,5239,TLKM.JK,2025-12-24,3480.0,3500.0,3440.0,3460.0,30919800,2026-01-01 15:07:38.819122,3454.0,3546.0,3465.0,-0.002882,37.313433
5239,5240,TLKM.JK,2025-12-29,3480.0,3530.0,3460.0,3490.0,53079900,2026-01-01 15:07:38.819122,3460.0,3542.0,3475.8,0.008671,33.333333
5240,5241,TLKM.JK,2025-12-30,3490.0,3510.0,3480.0,3480.0,48019100,2026-01-01 15:07:38.819122,3474.0,3540.5,3487.8,-0.002865,36.842105


In [10]:
df[["ma_5", "ma_20", "ma_50", "daily_return", "rsi"]].isna().sum()

ma_5             4
ma_20           19
ma_50           49
daily_return     1
rsi             13
dtype: int64